### google

In [1]:
import feedparser
import ssl
import urllib.request

In [2]:
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
corps = set()

keyword = '오늘날씨'

rss_url = f"https://news.google.com/rss/search?q={keyword}&hl=ko&gl=KR&ceid=KR:ko"
# rss_url = f"https://news.google.com/rss/search?q={keyword}"
feed = feedparser.parse(rss_url)

results = []
for entry in feed.entries:
    result = {
        'title': entry.title,
        'link': entry.link,
        'published': entry.published,
        'corp' : entry.title.split(' - ')[1]
    }
    results.append(result)
    corps.add(entry.title.split(' - ')[1])

print(len(results))

100


In [4]:
results

[{'title': '[오늘날씨] 전국 대체로 맑아···낮 최고기온 18~26도 - 뉴스핌',
  'link': 'https://news.google.com/rss/articles/CBMiMGh0dHBzOi8vd3d3Lm5ld3NwaW0uY29tL25ld3Mvdmlldy8yMDI0MDUwOTAwMDAxOdIBAA?oc=5',
  'published': 'Wed, 08 May 2024 21:06:00 GMT',
  'corp': '뉴스핌'},
 {'title': '[오늘 날씨 START 5/9] - YTN',
  'link': 'https://news.google.com/rss/articles/CBMiMWh0dHBzOi8vd3d3Lnl0bi5jby5rci9fbG4vMDEwOF8yMDI0MDUwOTA1NDYyMjY3NDXSAUNodHRwczovL20ueXRuLmNvLmtyL25ld3Nfdmlldy5hbXAucGhwP3BhcmFtPTAxMDhfMjAyNDA1MDkwNTQ2MjI2NzQ1?oc=5',
  'published': 'Wed, 08 May 2024 20:46:00 GMT',
  'corp': 'YTN'},
 {'title': '[날씨] 오늘 낮과 밤 기온차 커…해안가 강한 너울 주의 - 연합뉴스TV',
  'link': 'https://news.google.com/rss/articles/CBMiNmh0dHBzOi8vbS55b25oYXBuZXdzdHYuY28ua3IvbmV3cy9NWUgyMDI0MDUwODAxMDcwMDY0MdIBAA?oc=5',
  'published': 'Wed, 08 May 2024 05:52:09 GMT',
  'corp': '연합뉴스TV'},
 {'title': '[오늘날씨] “쾌청하다”… 전국 맑고 낮 최고 26도까지 - 천지일보',
  'link': 'https://news.google.com/rss/articles/CBMiOmh0dHBzOi8vd3d3Lm5ld3Njai5jb20vbmV3cy9hcnRpY2xlVmlldy5odG1s

In [5]:
links = [val['link'] for val in results]
links[:2]

['https://news.google.com/rss/articles/CBMiMGh0dHBzOi8vd3d3Lm5ld3NwaW0uY29tL25ld3Mvdmlldy8yMDI0MDUwOTAwMDAxOdIBAA?oc=5',
 'https://news.google.com/rss/articles/CBMiMWh0dHBzOi8vd3d3Lnl0bi5jby5rci9fbG4vMDEwOF8yMDI0MDUwOTA1NDYyMjY3NDXSAUNodHRwczovL20ueXRuLmNvLmtyL25ld3Nfdmlldy5hbXAucGhwP3BhcmFtPTAxMDhfMjAyNDA1MDkwNTQ2MjI2NzQ1?oc=5']

In [6]:
results[2]

{'title': '[날씨] 오늘 낮과 밤 기온차 커…해안가 강한 너울 주의 - 연합뉴스TV',
 'link': 'https://news.google.com/rss/articles/CBMiNmh0dHBzOi8vbS55b25oYXBuZXdzdHYuY28ua3IvbmV3cy9NWUgyMDI0MDUwODAxMDcwMDY0MdIBAA?oc=5',
 'published': 'Wed, 08 May 2024 05:52:09 GMT',
 'corp': '연합뉴스TV'}

In [7]:
import requests
from bs4 import BeautifulSoup, Comment
import feedparser
from newspaper import Article
import datetime
from fake_useragent import UserAgent


class NewsCrawler():
    def __init__(self):
        self.user_agent = UserAgent()
        self.headers = None

    def extract_tag(self, item, tag):
        [elem.extract() for elem in item(tag)]


    def extract_comments(self, item):
        for element in item(text=lambda text: isinstance(text, Comment)):
            element.extract()
            
    def get_headers(self, shuffle_header=False):
        if self.headers is None or shuffle_header:
            self.headers = {
                'User-Agent': self.user_agent.random,
            }
        return self.headers


    def crawl_url(self, url, shuffle_header=False, language='ko'):
        try:
            # article title
            article = Article(url, language=language)
            article.download()
            article.parse()
            if not article.url:
                print("[ERROR] no url found..")
                return None        

            data = {}
            data['title'] = article.title
            data['image'] = article.meta_img
            data['crawl_url'] = url
            data['url'] = article.url

            print(article.url)
            if article.publish_date:
                data['published_at'] = article.publish_date
            else:
                data['published_at'] = datetime.datetime.now()

            res = requests.get(url, headers=self.get_headers(shuffle_header=shuffle_header))
            soup = BeautifulSoup(res.text, 'html.parser')

            for script in soup(['script', 'style']):
                script.decompose()

            body = soup.find("div", itemprop="articleBody")

            if body is None:
                body = soup.select('#articleBodyContents')
                if len(body) > 0:
                    body = body[0]
                    self.extract_comments(body)
                    self.extract_tag(body, 'img')
                    self.extract_tag(body, 'div')
                    body = str(body)
                else:
                    body = article.text
                    body = body.replace('\n','<br>')
            else:
                self.extract_comments(body)
                self.extract_tag(body, 'img')
                self.extract_tag(body, 'div')
                body = str(body)

            data['text'] = article.text
            data['body'] = body
            data['language'] = language

        except Exception as e:
            print(e)
            data = None

        return data
    
    def crawl_rss(self, rss_url, shuffle_header=False, language='ko'):
        feed = feedparser.parse(rss_url)
        data_list = []
        for post in feed.entries:
            if post.link:
                data_list.append(self.crawl_url(post.link, shuffle_header, language))

        return data_list

In [8]:
sample_url = """https://news.google.com/rss/articles/CBMiMGh0dHBzOi8vd3d3Lm5ld3NwaW0uY29tL25ld3Mvdmlldy8yMDI0MDUwODAwMDAxOdIBMWh0dHBzOi8vbS5uZXdzcGltLmNvbS9uZXdzYW1wL3ZpZXcvMjAyNDA1MDgwMDAwMTk?oc=5
"""

c = NewsCrawler()
# tmp = c.crawl_url(sample_url)


In [9]:
import pandas as pd

In [10]:
url_df = pd.DataFrame(columns=['title','image','crawl_url','url','published_at','text','body'])
url_df


,title,image,crawl_url,url,published_at,text,body


In [11]:

c = NewsCrawler()

In [12]:
# for r in results:
#     tmp = c.crawl_url(r['link'])
#     length = len(url_df)+1
    
#     url_df.loc[length, 'title'] = tmp['title']
#     url_df.loc[length, 'image'] = tmp['image']
#     url_df.loc[length, 'crawl_url'] = tmp['crawl_url']
#     url_df.loc[length, 'url'] = tmp['url']
#     url_df.loc[length, 'published_at'] = tmp['published_at']
#     url_df.loc[length, 'text'] = tmp['text']
#     url_df.loc[length, 'body'] = tmp['body']
    
# url_df

In [13]:
import re


In [14]:
# custom_words = ['기자', '일보', '앵커']   


In [15]:
# for idx, row in url_df.loc[:1].iterrows():
#     url = row['url']
#     print(row['text'])
#     text = row['text'].replace('\n', '')
#     pattern1 = r'\b(?:' + '|'.join(list(corps)) + r')\b'
#     preprocessed_text = re.sub(pattern1, '', text)
#     pattern2 = r'[^\w\s,~.\'"]'
#     preprocessed_text = re.sub(pattern2, '', preprocessed_text)
#     pattern3 = r'\b(?:\w+ 앵커|\w+ 기자|\w+일보)\b'
#     preprocessed_text = re.sub(pattern3, '', preprocessed_text)

#     print(preprocessed_text)

### keyword 추출

In [16]:
from mecab import MeCab as mecab

In [17]:
mecab = mecab()

## mecab-co-dic 품사 태그

    - NNG : 일반명사
    - NNP : 고유명사
    - NNB : 의존명사
    - NNBC : 단위를 나타내는 명사
    - NR : 수사
    - NP : 대명사

In [18]:
query = '내일 날씨 알려줘'
print(f" 원 쿼리 -> {query}")
mecab_result = mecab.pos(query)
print(f" mecab pos tagging -> {mecab_result}")

result = []
combined_word = ''

cur = 0
prev_pos = None
while cur < len(mecab_result):
    word, pos = mecab_result[cur]
    if pos == 'SN':
        combined_word += word
        nxt = cur + 1
        while nxt < len(mecab_result) and mecab_result[nxt][1] == 'SN':
            combined_word += mecab_result[nxt][0]
            nxt += 1
            
        while nxt < len(mecab_result) and mecab_result[nxt][1] in ['NNG', 'NNP', 'NNB', 'NNBC', 'NR', 'NP']:
            if mecab_result[nxt-1][1] in ['NNG', 'NNP', 'NNB', 'NNBC', 'NR', 'NP']:
                break
            combined_word += mecab_result[nxt][0]
            nxt += 1
        result.append(combined_word)
        combined_word = ''
        cur = nxt
    elif pos in ['MAG', 'NNG', 'NNP', 'NNB', 'NNBC', 'NR', 'NP']:
        combined_word = word
        result.append(combined_word)
        nxt = cur + 1
        combined_word = ''
        cur = nxt
    else:
        cur += 1

print(f" 키워드 추출 -> {result}")

 원 쿼리 -> 내일 날씨 알려줘
 mecab pos tagging -> [('내일', 'MAG'), ('날씨', 'NNG'), ('알려줘', 'VV+EC+VX+EC')]
 키워드 추출 -> ['내일', '날씨']


### 검색어 가져와서 rss 검색

In [19]:
keyword = '+'.join(result)
print(keyword)

내일+날씨


In [20]:
corps = set()

rss_url = f"https://news.google.com/rss/search?q={keyword}&hl=ko&gl=KR&ceid=KR:ko"
# rss_url = f"https://news.google.com/rss/search?q={keyword}"
feed = feedparser.parse(rss_url)

results = []
for entry in feed.entries:
    result = {
        'title': entry.title,
        'link': entry.link,
        'published': entry.published,
        'corp' : entry.title.split(' - ')[1]
    }
    results.append(result)
    corps.add(entry.title.split(' - ')[1])

print(len(results))

100


In [21]:
url_call_df = pd.DataFrame(results)
url_call_df

,title,link,published,corp
0,[날씨] 내일 전국 맑고 일교차 커…충청·남부 짙은 안개 - 연합뉴스TV,https://news.google.com/rss/articles/CBMiOGh0d...,"Wed, 08 May 2024 14:09:48 GMT",연합뉴스TV
1,"[내일날씨] 아침 기온 10도 안팎, 낮 최고 26도...큰 일교차 주의 - 이투데이",https://news.google.com/rss/articles/CBMiKmh0d...,"Wed, 08 May 2024 11:24:00 GMT",이투데이
2,[내일날씨] 전국 대체로 맑아...낮 최고기온 27도까지 - 아주경제,https://news.google.com/rss/articles/CBMiLmh0d...,"Wed, 08 May 2024 10:45:40 GMT",아주경제
3,"[내일날씨] 목요일, 전국 맑고 낮부터 기온 올라... '큰 일교차 여전' - 충청신문",https://news.google.com/rss/articles/CBMiOmh0d...,"Wed, 08 May 2024 11:01:00 GMT",충청신문
4,[내일날씨] 10도 이상 일교차…낮에는 27도까지도 - 뉴스핌,https://news.google.com/rss/articles/CBMiMGh0d...,"Wed, 08 May 2024 07:28:00 GMT",뉴스핌
...,...,...,...,...
95,[날씨] 내일까지 전국 비...제주도·남해안 돌풍 동반 - YTN,https://news.google.com/rss/articles/CBMiMWh0d...,"Sun, 05 May 2024 10:20:00 GMT",YTN
96,[내일날씨] 전국 흐리고 비…낮최고 15~26도 - 매일신문,https://news.google.com/rss/articles/CBMiNGh0d...,"Sun, 05 May 2024 01:00:00 GMT",매일신문
97,[내일 날씨] 새벽까지 충청권과 남부지방 중심 가끔 비…동해안 너울 주의 - 뉴스웍스,https://news.google.com/rss/articles/CBMiPmh0d...,"Tue, 07 May 2024 09:01:19 GMT",뉴스웍스
98,[내일 날씨] 전국 대부분 비…강풍 조심하세요 - 머니투데이,https://news.google.com/rss/articles/CBMiN2h0d...,"Sun, 05 May 2024 08:49:25 GMT",머니투데이


### ML 기반 유사도 높은 문서 3개 뽑아오기

In [22]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
from sentence_transformers import SentenceTransformer

/Users/geonheekim/Desktop/rss/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
model = SentenceTransformer('sentence-transformers/stsb-xlm-r-multilingual')

/Users/geonheekim/Desktop/rss/venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/geonheekim/Desktop/rss/venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### ML 기반 유사도 높은 문서 5개 가져오기

In [24]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
from sentence_transformers import SentenceTransformer

In [25]:
model = SentenceTransformer('sentence-transformers/stsb-xlm-r-multilingual')

In [26]:
train_data = url_call_df.copy()

In [27]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.title), axis = 1)

In [29]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [31]:
train_data

,title,link,published,corp,embedding,score
0,[날씨] 내일 전국 맑고 일교차 커…충청·남부 짙은 안개 - 연합뉴스TV,https://news.google.com/rss/articles/CBMiOGh0d...,"Wed, 08 May 2024 14:09:48 GMT",연합뉴스TV,"[-0.392392, -0.10396661, 1.0214683, -0.417209,...",0.424596
1,"[내일날씨] 아침 기온 10도 안팎, 낮 최고 26도...큰 일교차 주의 - 이투데이",https://news.google.com/rss/articles/CBMiKmh0d...,"Wed, 08 May 2024 11:24:00 GMT",이투데이,"[0.06254994, 0.03358691, 0.9233255, -0.0015778...",0.918382
2,[내일날씨] 전국 대체로 맑아...낮 최고기온 27도까지 - 아주경제,https://news.google.com/rss/articles/CBMiLmh0d...,"Wed, 08 May 2024 10:45:40 GMT",아주경제,"[-0.1054789, -0.08803947, 0.94445056, 0.088917...",0.762216
3,"[내일날씨] 목요일, 전국 맑고 낮부터 기온 올라... '큰 일교차 여전' - 충청신문",https://news.google.com/rss/articles/CBMiOmh0d...,"Wed, 08 May 2024 11:01:00 GMT",충청신문,"[-0.12550868, 0.028314134, 0.76828855, -0.0701...",0.706851
4,[내일날씨] 10도 이상 일교차…낮에는 27도까지도 - 뉴스핌,https://news.google.com/rss/articles/CBMiMGh0d...,"Wed, 08 May 2024 07:28:00 GMT",뉴스핌,"[0.16616865, -0.09602907, 0.92959553, -0.07314...",0.814835
...,...,...,...,...,...,...
95,[날씨] 내일까지 전국 비...제주도·남해안 돌풍 동반 - YTN,https://news.google.com/rss/articles/CBMiMWh0d...,"Sun, 05 May 2024 10:20:00 GMT",YTN,"[-0.37098944, -0.13383783, 1.085356, -0.396284...",0.430279
96,[내일날씨] 전국 흐리고 비…낮최고 15~26도 - 매일신문,https://news.google.com/rss/articles/CBMiNGh0d...,"Sun, 05 May 2024 01:00:00 GMT",매일신문,"[-0.16228098, 0.15558484, 0.6723706, -0.099671...",0.684992
97,[내일 날씨] 새벽까지 충청권과 남부지방 중심 가끔 비…동해안 너울 주의 - 뉴스웍스,https://news.google.com/rss/articles/CBMiPmh0d...,"Tue, 07 May 2024 09:01:19 GMT",뉴스웍스,"[0.10756793, -0.055630166, 0.87209487, -0.0230...",0.838599
98,[내일 날씨] 전국 대부분 비…강풍 조심하세요 - 머니투데이,https://news.google.com/rss/articles/CBMiN2h0d...,"Sun, 05 May 2024 08:49:25 GMT",머니투데이,"[0.08016775, 0.0070518428, 0.821678, 0.0332551...",0.865254


In [33]:
embedding = model.encode(query)
train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
sim_result_df = train_data.sort_values(by='score', ascending=False)[:5].reset_index(drop=True)
sim_result_df.loc[:5]

,title,link,published,corp,embedding,score
0,[날씨] 퇴근길 곳곳 비...내일 큰 일교차 주의 - YTN,https://news.google.com/rss/articles/CBMiMWh0d...,"Tue, 07 May 2024 08:28:00 GMT",YTN,"[0.08333481, -0.00431844, 0.95675135, 0.029431...",0.954758
1,[내일날씨] 전국 흐리고 비…수도권 강수량 20~60㎜ - 이투데이,https://news.google.com/rss/articles/CBMiKmh0d...,"Sun, 05 May 2024 11:05:00 GMT",이투데이,"[0.09067341, 0.0025862963, 0.9752792, -0.02546...",0.926022
2,[날씨] 퇴근길 쌀쌀...내일 점차 맑아져 - YTN,https://news.google.com/rss/articles/CBMiMWh0d...,"Tue, 07 May 2024 09:50:00 GMT",YTN,"[0.16286604, -0.05778117, 1.2462864, 0.0798728...",0.922108
3,[내일날씨] 전국 대체로 맑아...낮 최고기온 27도까지 - 네이트뷰,https://news.google.com/rss/articles/CBMiKmh0d...,"Wed, 08 May 2024 11:12:31 GMT",네이트뷰,"[0.14447114, -0.016954724, 1.0028929, -0.01145...",0.920705
4,"[내일날씨] 아침 기온 10도 안팎, 낮 최고 26도...큰 일교차 주의 - 이투데이",https://news.google.com/rss/articles/CBMiKmh0d...,"Wed, 08 May 2024 11:24:00 GMT",이투데이,"[0.06254994, 0.03358691, 0.9233255, -0.0015778...",0.918382


In [99]:
content_df = pd.DataFrame()

In [100]:
for idx, row in sim_result_df[:10].iterrows():
    tmp = c.crawl_url(row['link'])
    length = len(content_df)+1
    
    try :
        content_df.loc[length, 'title'] = tmp['title']
        content_df.loc[length, 'url'] = tmp['url']
        content_df.loc[length, 'published_at'] = tmp['published_at']
        content_df.loc[length, 'text'] = tmp['text'].replace('\n', '')
    except:
        pass
    
content_df

https://news.google.com/rss/articles/CBMiMWh0dHBzOi8vd3d3Lnl0bi5jby5rci9fbG4vMDEwOF8yMDI0MDUwNzE3Mjg1Nzc3NDbSAUNodHRwczovL20ueXRuLmNvLmtyL25ld3Nfdmlldy5hbXAucGhwP3BhcmFtPTAxMDhfMjAyNDA1MDcxNzI4NTc3NzQ2?oc=5
https://news.google.com/rss/articles/CBMiKmh0dHBzOi8vd3d3LmV0b2RheS5jby5rci9uZXdzL3ZpZXcvMjM1NzExMdIBAA?oc=5
https://news.google.com/rss/articles/CBMiMWh0dHBzOi8vd3d3Lnl0bi5jby5rci9fbG4vMDEwOF8yMDI0MDUwNzE4NTAwMzcxNTPSAUNodHRwczovL20ueXRuLmNvLmtyL25ld3Nfdmlldy5hbXAucGhwP3BhcmFtPTAxMDhfMjAyNDA1MDcxODUwMDM3MTUz?oc=5
https://news.google.com/rss/articles/CBMiKmh0dHBzOi8vdmlldy5uYXRlLmNvbS9lY29ub215L3ZpZXcvMTg5ODQxL9IBAA?oc=5
https://news.google.com/rss/articles/CBMiKmh0dHBzOi8vd3d3LmV0b2RheS5jby5rci9uZXdzL3ZpZXcvMjM1ODEyMtIBAA?oc=5


,title,url,published_at,text
1,[날씨] 퇴근길 곳곳 비...내일 큰 일교차 주의,https://news.google.com/rss/articles/CBMiMWh0d...,2024-05-07 17:28:00+09:00,"AD퇴근길, 여전히 곳곳에 비가 내리며, 서늘함이 감돌고 있습니다.내일 새벽까지 충..."
2,[내일날씨] 전국 흐리고 비…수도권 강수량 20~60㎜,https://news.google.com/rss/articles/CBMiKmh0d...,2024-05-05 20:05:00+09:00,▲기상청 홈페이지월요일인 6일은 흐리고 비 오는 날씨가 이어지겠다.5일부터 이틀간 ...
3,[날씨] 퇴근길 쌀쌀...내일 점차 맑아져,https://news.google.com/rss/articles/CBMiMWh0d...,2024-05-07 18:50:00+09:00,"AD현재 수도권과 충청, 남부 내륙을 중심으로 비가 내리고 있습니다.비 구름이 남동..."
4,[내일날씨] 전국 대체로 맑아…낮 최고기온 27도까지,https://news.google.com/rss/articles/CBMiKmh0d...,2024-05-08 20:12:31+09:00,지난 8일 경기도 수원시에 있는 화성(華城) 창룡문 위 [사진=연합뉴스]목요일인 9...
5,"[내일날씨] 아침 기온 10도 안팎, 낮 최고 26도...큰 일교차 주의",https://news.google.com/rss/articles/CBMiKmh0d...,2024-05-08 20:24:00+09:00,(고이란 기자 photoeran@)9일 목요일은 전국이 맑은 가운데 내륙 중심으로 ...


### LLM 요약

In [101]:
from dotenv import load_dotenv

load_dotenv()

True

In [102]:
import json

In [103]:
result_data_json = content_df.to_json(orient='records', force_ascii=False)
result_data_json

'[{"title":"[날씨] 퇴근길 곳곳 비...내일 큰 일교차 주의","url":"https:\\/\\/news.google.com\\/rss\\/articles\\/CBMiMWh0dHBzOi8vd3d3Lnl0bi5jby5rci9fbG4vMDEwOF8yMDI0MDUwNzE3Mjg1Nzc3NDbSAUNodHRwczovL20ueXRuLmNvLmtyL25ld3Nfdmlldy5hbXAucGhwP3BhcmFtPTAxMDhfMjAyNDA1MDcxNzI4NTc3NzQ2?oc=5","published_at":1715070480000,"text":"AD퇴근길, 여전히 곳곳에 비가 내리며, 서늘함이 감돌고 있습니다.내일 새벽까지 충청 이남을 중심으로는 5∼30mm의 비가 조금 더 이어진 뒤, 그치겠습니다.어버이날인 내일, 하늘이 차차 개겠지만, 제주도는 오후까지 비가 내리겠습니다.또 경기 북동부와 강원 북부에는 빗방울이 떨어지는 곳이 있겠습니다.내일 아침 기온은 오늘보다, 또 예년보다 낮아 서늘하겠습니다.서울 11도, 대전 10도, 부산 12도에서 출발하겠습니다.내일 낮 기온은 20도 안팎까지 올라, 일교차가 크게 벌어지겠습니다.서울 21도, 대전 21도, 광주 22도가 예상됩니다.주 후반까지는 대체로 맑고 포근한 봄 날씨가 이어지겠고요,다가오는 주말에는 또다시 전국 곳곳에 비가 내릴 전망입니다.날씨였습니다.[저작권자(c) YTN 무단전재, 재배포 및 AI 데이터 활용 금지]"},{"title":"[내일날씨] 전국 흐리고 비…수도권 강수량 20~60㎜","url":"https:\\/\\/news.google.com\\/rss\\/articles\\/CBMiKmh0dHBzOi8vd3d3LmV0b2RheS5jby5rci9uZXdzL3ZpZXcvMjM1NzExMdIBAA?oc=5","published_at":1714907100000,"text":"▲기상청 홈페이지월요일인 6일은 흐리고 비 오는 날씨가 이어지겠다.5일부터 이틀간 예상 강수량은 서울·인천·경기·강원 내륙·산지·충북·

### LLM (Retrieval)

In [104]:
from langchain_community.document_loaders import JSONLoader

In [105]:
from pprint import pprint

In [106]:
result_json = json.loads(result_data_json)
pprint(result_json)

[{'published_at': 1715070480000,
  'text': 'AD퇴근길, 여전히 곳곳에 비가 내리며, 서늘함이 감돌고 있습니다.내일 새벽까지 충청 이남을 중심으로는 5∼30mm의 '
          '비가 조금 더 이어진 뒤, 그치겠습니다.어버이날인 내일, 하늘이 차차 개겠지만, 제주도는 오후까지 비가 내리겠습니다.또 '
          '경기 북동부와 강원 북부에는 빗방울이 떨어지는 곳이 있겠습니다.내일 아침 기온은 오늘보다, 또 예년보다 낮아 '
          '서늘하겠습니다.서울 11도, 대전 10도, 부산 12도에서 출발하겠습니다.내일 낮 기온은 20도 안팎까지 올라, 일교차가 '
          '크게 벌어지겠습니다.서울 21도, 대전 21도, 광주 22도가 예상됩니다.주 후반까지는 대체로 맑고 포근한 봄 날씨가 '
          '이어지겠고요,다가오는 주말에는 또다시 전국 곳곳에 비가 내릴 전망입니다.날씨였습니다.[저작권자(c) YTN 무단전재, '
          '재배포 및 AI 데이터 활용 금지]',
  'title': '[날씨] 퇴근길 곳곳 비...내일 큰 일교차 주의',
  'url': 'https://news.google.com/rss/articles/CBMiMWh0dHBzOi8vd3d3Lnl0bi5jby5rci9fbG4vMDEwOF8yMDI0MDUwNzE3Mjg1Nzc3NDbSAUNodHRwczovL20ueXRuLmNvLmtyL25ld3Nfdmlldy5hbXAucGhwP3BhcmFtPTAxMDhfMjAyNDA1MDcxNzI4NTc3NzQ2?oc=5'},
 {'published_at': 1714907100000,
  'text': '▲기상청 홈페이지월요일인 6일은 흐리고 비 오는 날씨가 이어지겠다.5일부터 이틀간 예상 강수량은 서울·인천·경기·강원 '
          '내륙·산지·충북·대구·경북·울릉도·독도 20∼60㎜, 서해5도·강원 동해안 10∼40㎜다.광주·전남·전북은 '
          '30∼8

In [127]:
loader = JSONLoader(
    file_path=result_json,
    jq_schema='',
    text_content=False,
    json_lines=True)

data = loader.load()

TypeError: expected str, bytes or os.PathLike object, not list

### LLM (요약) 

In [107]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

In [108]:
model = ChatOpenAI(model='gpt-3.5-turbo',
                   temperature=0)

In [109]:
class SummaryObject(BaseModel):
    title: str = Field(description="본문의 title")
    url: str = Field(description="본문에 해당하는 url")
    summary: str = Field(description="본문을 요약한 텍스트")

In [110]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import ConversationChain

In [111]:
parser = JsonOutputParser(pydantic_object=SummaryObject)

In [112]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert summarizer."
            "Summarizes text content related to a given query."
            ),
            (
            "user",
            "Please summarize the sentences according to the following requests."
            "\nRequest:\n"
            "1. Summarizes only content related to the provided query in Korean."
            "2. Provide a URL related to the referenced content, but please mention only the URL address."
            "3. Please translate the summary into Korean."
            "\n\nQuery: {query}\n\nContext: {context}\n\nSummary:",
                    ),
    ]
)
chain = prompt | llm | parser
response= chain.invoke({"query": query, "context" : result_json})

response.content

OutputParserException: Invalid json output: URL addresses related to the referenced content:
1. https://news.google.com/rss/articles/CBMiMWh0dHBzOi8vd3d3Lnl0bi5jby5rci9fbG4vMDEwOF8yMDI0MDUwNzE3Mjg1Nzc3NDbSAUNodHRwczovL20ueXRuLmNvLmtyL25ld3Nfdmlldy5hbXAucGhwP3BhcmFtPTAxMDhfMjAyNDA1MDcxNzI4NTc3NzQ2?oc=5

내일 날씨 알려줘에 관련된 내용을 한국어로 요약하면:
퇴근길에는 곳곳에 비가 내리고 있으며, 내일은 큰 일교차에 주의해야 합니다. 충청 이남 지역을 중심으로 비가 이어지다가 그칠 예정이며, 서울은 11도, 대전은 10도, 부산은 12도에서 출발할 것으로 예상됩니다. 내일 낮 기온은 20도 안팎까지 올라 일교차가 커지겠으며, 주 후반까지는 맑고 포근한 봄 날씨가 이어지다가 주말에는 전국 곳곳에 비가 내릴 전망입니다.